<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/07_LogisticRegression_3ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

The file contains 3 version of computation:

1.   Having the real data, consider a part as 1 and another as 0
2.   Having the real data, consider a part as 1 and another as 0 of the same size
3.   Consider the real data as 1 and create unattested colexifications considered 0

In the cases 1 and 2, the Regression model tries to predict the most probable colexifications; in the case 3 it tries to predict whether a pair is at all a colexification.

The computation are executed only on one dataframe and not on every cycle of the bootstrap. This is a testing!

### Version 1
Use a model to predict binary value on real data

In [ ]:
from google.colab import files
import pickle
import pandas as pd

In [ ]:
!gdown --id 1tu36NAL3MyHMkajE7wEDffbF0OI5fidY #listBoot
# bootstrappping with 3 as number of cicle

Downloading...
From: https://drive.google.com/uc?id=1tu36NAL3MyHMkajE7wEDffbF0OI5fidY
To: /content/listBoot.txt
6.36MB [00:00, 20.2MB/s]


In [ ]:
with open("listBoot.txt", "rb") as fp:   # Unpickling
  lBoot = pickle.load(fp)

In [ ]:
lChoosen = lBoot[1] # to avoid problems when changing number
C1 = [i[0][0] for i in lChoosen]
C2 = [i[0][1] for i in lChoosen]
df = pd.DataFrame(lChoosen, columns=['Colex_pair', 'Prob']) # to save in df
df ['Concept1'] = C1
df['Concept2'] = C2

In [ ]:
df.head(2)

,Colex_pair,Prob,Concept1,Concept2
0,"(a little, what)",0.000044,a little,what
1,"(above, black)",0.000153,above,black


In [ ]:
df = df.sort_values(by=['Prob'])
df = df.reset_index(drop = True)

Set boolean column using histogram value: the top 2% is considered 1, the remaining, 0



In [ ]:
totN = df.shape[0]
selected = round(totN * 0.02)
df ['Colex_bool'] = 0
df.loc[( totN - selected) : ,'Colex_bool'] = 1

In [ ]:
df.groupby('Colex_bool').size()

Colex_bool
0    42871
1      875
dtype: int64

In [ ]:
df.head(2)

,Colex_pair,Prob,Concept1,Concept2,Colex_bool
0,"(chew, ring)",0.000016,chew,ring,0
1,"(wages, chop)",0.000016,wages,chop,0


## Obtain Cosine values from other file!!

The current file doesn't contain the FastText model to calculate cosines. It is necessary to obtain these values from another file.

In [ ]:
df.to_csv('df_supp.csv', index = False)

In [ ]:
df = pd.read_csv('df_cosines.csv')

In [ ]:
df.head(2)

,Colex_pair,Prob,Concept1,Concept2,Colex_bool,Cosine
0,"('chew', 'ring')",0.000016,chew,ring,0,0.131701
1,"('wages', 'chop')",0.000016,wages,chop,0,0.136339


Try sklearn Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
cols = ['Cosine']
X = df[cols]
y = df['Colex_bool']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.98


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8567
           1       0.00      0.00      0.00       183

    accuracy                           0.98      8750
   macro avg       0.49      0.50      0.49      8750
weighted avg       0.96      0.98      0.97      8750



The model assign 0 to all the element, obtaining a high accuracy just because we have an highly unbalanced dataset!

### Version 2
Try to select the same amount of value of 1 for 0: balance with real data.

In [ ]:
g = df.groupby('Colex_bool')
g = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))

In [ ]:
cols = ['Cosine']
X = g[cols]
y = g['Colex_bool']

for i in range(10):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  logreg = LogisticRegression()
  logreg.fit(X_train, y_train)
  y_pred = logreg.predict(X_test)
  print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))


Accuracy of logistic regression classifier on test set: 0.79
Accuracy of logistic regression classifier on test set: 0.78
Accuracy of logistic regression classifier on test set: 0.82
Accuracy of logistic regression classifier on test set: 0.81
Accuracy of logistic regression classifier on test set: 0.82
Accuracy of logistic regression classifier on test set: 0.79
Accuracy of logistic regression classifier on test set: 0.82
Accuracy of logistic regression classifier on test set: 0.81
Accuracy of logistic regression classifier on test set: 0.80
Accuracy of logistic regression classifier on test set: 0.80


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.78      0.78       179
           1       0.77      0.78      0.77       171

    accuracy                           0.78       350
   macro avg       0.78      0.78      0.78       350
weighted avg       0.78      0.78      0.78       350



This time the value is realible, and higher than random (0.5)

### Version 3
Create unattested colexifications ("fake colexifications")

In [ ]:
df_fake = df.copy()

In [ ]:
df_fake['Colex_bool'] = 1 # all the existing colexifications are real

In [ ]:
df_fake.groupby('Colex_bool').size()

Colex_bool
1    43746
dtype: int64

In [ ]:
real_col = list(df_fake['Colex_pair'])

In [ ]:
real_col = [eval(t) for t in real_col] # convert string to tuple

In [ ]:
# create list with single items
real_col_l = [item for t in real_col for item in t]
real_col_l = list(set(real_col_l)) # delete duplicates

In [ ]:
len(real_col_l)

2159

In [ ]:
import random

We create unattested colexifications randomly picking elements from the list, and keeping only those that are not in the original dataframe.

In [ ]:
fake_col = []
for i in range(totN):
  el1 = random.choice(real_col_l)
  el2 = random.choice(real_col_l)
  while el1 == el2:
    el2 = random.choice(real_col_l)
  supp_t = (el1, el2)
  if supp_t not in set(real_col): # take only tuple that is not in real ones --> size not the same but almost
    fake_col.append(supp_t)
    # fix difference in number i--

In [ ]:
len(fake_col)

43311

In [ ]:
totN

43746

The number of unattested colexifications is less than the attested ones, but still, the resulting dataframe is almost balanced.

In [ ]:
df_fake_supp = pd.DataFrame(fake_col, columns =['Concept1', 'Concept2'])
df_fake_supp['Colex_pair'] = fake_col
df_fake_supp['Colex_bool'] = 0

In [ ]:
df_fake_supp.head(2)

,Concept1,Concept2,Colex_pair,Colex_bool
0,bait,ten,"(bait, ten)",0
1,thursday,weather,"(thursday, weather)",0


In [ ]:
df_fake_supp.to_csv('df_supp.csv', index = False)

In [ ]:
files.download('df_supp.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Import the dataframe with cosine values

In [ ]:
df_fake_cosine = pd.read_csv('df_cosines.csv')

In [ ]:
df_fake_cosine.head(2)

,Concept1,Concept2,Colex_pair,Colex_bool,Cosine
0,bait,ten,"('bait', 'ten')",0,0.050746
1,thursday,weather,"('thursday', 'weather')",0,0.224283


In [ ]:
df_fake.head(2)

,Colex_pair,Prob,Concept1,Concept2,Colex_bool,Cosine
0,"('chew', 'ring')",0.000016,chew,ring,1,0.131701
1,"('wages', 'chop')",0.000016,wages,chop,1,0.136339


In [ ]:
df_fake_cosine = df_fake_cosine.rename(columns={"cosine_sim": "Cosine"})

In [ ]:
df_fake = df_fake.append(df_fake_cosine)

Data is now ready

In [ ]:
cols = ['Cosine']
X = df_fake[cols]
y = df_fake['Colex_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.64


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.70      0.66      8734
           1       0.66      0.57      0.61      8678

    accuracy                           0.64     17412
   macro avg       0.64      0.64      0.64     17412
weighted avg       0.64      0.64      0.64     17412



Also this time, the result is higher than random.